## Main Scan Operations

### Sequential Scan
This is the simplest way of fetching data from a table, it scans through every page of data sequentially.

#### Use Cases
- It's the only option
- The query required a large proportion of the table

### Index Scan
Traverse the index to find the matching predicate rows and then fetch them from the table -> two searches per row.


#### Use Cases
- A small portion of the table is required

### Index Only Scan
Happens when the actual data needed is found completely inside the index. So we just traverse the index and take the data from there. It's much more preferred than regular index scan because it avoids the overhead of searching the actual table.

#### Use Cases
- A small portion of the table is required
- Index is supporting (mainly due to MVCC and Data Structure)
- All the data exists in the index

### Bitmap Index Scan
Traverse the index and create a bitmap of the index values to match with the table values later on. If the bitmap structure is getting too big, it can switch to `lossy` strategy which just keeps  data to a file This stage only creates the bitmap which is used in later steps.

#### Use Cases
- A medium proportion of the table is required
- Index is present
- When index order is not an issue -> no matching ORDER BY clause

### Bitmap Heap Scan
Applying the bitmap created by `Bitmap Index Scan` (maybe combined with `BitmapAnd` or `BitmapOr` operations) to actually fetch the rows from the table

#### Use Cases
- A bitmap structure is present

### CTE Scan

When using a CTE postgres can use it with 2 strategies:
1. Materialize the CTE and treat it like a temp table, done when the CTE is referenced multiple times 
    - Pros: single time execution
    - Cons: no optimization based on parent query, no propagation of restrictions
1. Not materialize the CTE so just treat it like a query tree (sometime as a sub query)
    - Pros: can optimize based on parent
    - Cons: will be calculated multiple times if needed

You can enforce a strategy by using `MATERIALIZED` or `NOT MATERIALIZED` keywords in CTE definition

## Main Join Operations

### Nested Loop Joins

#### Regular Nested Loop Join

Always estimated because it's always an option, but rarely used because more performant variations are usually available

<img src="./helpers/regular-nested-loop-join.png" alt="drawing" width="900"/>

#### Materialized Nested Loop Join

First materialize all the inner table (in memory or with spill) and than perform a regular nested loop join with cheaper inner table scan.

<img src="./helpers/materialize-nested-loop-join.png" alt="drawing" width="900"/>

#### Indexes Nested Loop Join (inner)

If there is an index on the inner table, it can be used to avoid the re-scanning of the inner table at every loop iteration, causing the loop to run only a single time!

<img src="./helpers/inner-indexed-nested-loop-join.png" alt="drawing" width="900"/>

#### Other Variations

1. Outer index nested loop join - If there is an index on the outer table, especially when it's content is relevant for the WHERE clause of the outer table, it can reduce the outer loop greatly. Of course if all of the outer table is needed it's pretty useless
1. Outer index with inner materialize
1. Outer index with inner index


<img src="./helpers/other-variation-nested-loop-join.png" alt="drawing" width="900"/>

### Merge Join
Only relevant when it's an equi-join, which means the join condition is based on `=` equality

#### Regular Merge Join
Sorts both inner and outer tables and then performs the join

<img src="./helpers/regular-merge-join.png" alt="drawing" width="900"/>

#### Materialized Merge Join
Merge join with materializing the inner table

<img src="./helpers/materialized-merge-join.png" alt="drawing" width="900"/>

#### Other Variations

1. Merge join with outer index scan
1. Materialized merge join with outer index scan
1. Indexed merge join with outer index scan

<img src="./helpers/other-variations-merge-join.png" alt="drawing" width="900"/>

### Hash Joins
Only relevant when it's an equi-join, which means the join condition is based on `=` equality

#### In Memory Hash Join
Happens when the inner table (hash target) size is 25% or less of the work_mem.

Steps:
1. Create a batch on work_mem
1. Insert the first tuple of the build table into the corresponding bucket of the batch by using hash function on join attribute
1. Insert remaining tuple of build table
1. Probe the first tuple of the probe table (check hash, than check complete equality since hash can be many to many) and join if equal
1. Probe the rest

Terms:
- Batch - hash table area
- Bucket - a single slot in the hash table
- Build table (inner) - the table which the hash is built on
- Probe table (outer) - the table which tuples are checked against teh build

<img src="./helpers/in-memory-hash-join-1.png" alt="drawing" width="900"/>
<img src="./helpers/in-memory-hash-join-2.png" alt="drawing" width="900"/>

#### Hybrid Skewed Hash Join
When all the tuples of the inner table can not be stored in memory (work_mem), a hybrid join is used which is basically splitting the hash table to multiple batches and dealing with them one at a time in memory.

This hash technique is based on a mix of grace hash join with classical hash join and some optimization

##### Grace Hash Join

**Problem**

Not enough memory to store the inner table and do the comparisons in memory

**Optimization**

Use less memory by partitioning the data on both sides and join step by step. 

<img src="./helpers/grace-hash-join.png" alt="drawing" width="500"/>

**Steps**
1. Partition by some partitioning hash on the join attribute both the inner and outer table and write all to disk
1. If the partitions are still too big it can be done recursively to partition another time
1. Fetch matching partitions for both tables and preform join
1. Loop over the fetch until all partitions are joined

##### Standard Hybrid Hash Join

**Problem**

We are not using all the memory possible (always a bad idea) + wasting time on writing the first partition on both sides to disk while we could just maintain it in memory.

**Optimization**

Mixing the grace hash join with the classical, by dividing the memory to two parts:
- Partition the data
- Save the first partition of the tables in memory, but make it the largest 

<img src="./helpers/hybrid-hash-join.png" alt="drawing" width="500"/>

Steps:
1. Partition the data
1. Carefully estimate how much batch partitions can fit in memory without the need to write them to disk
1. Write all batches to disk except the first x batches which could fit in memory
1. Perform the grace hash join algorithm as usual

##### Skewed Optimization

**Problem**

When joining two tuples from inner and outer table, the resulting row becomes *bigger*, so unfortunately the batch itself becomes bigger as well. This can cause movement of tuples between batches and result in OOM and other problems.

**Optimization**

Check the outer table statistics (MCV - Most Common Values) and make sure that the inner table tuples that will match the most outer table tuples will be in the first batch called skew

<img src="./helpers/skew-hash-join-optimization.png" alt="drawing" width="900"/>

## Main Other Operations

### Materialize
Bring some data into the local environment preferred to memory (work_mem) and when it's not enough to disk with spill